In [85]:
import pandas as pd

# Load the dataset
file_path = "../data/FE_no_outliers_dataset.csv" #"../data/ml-engineer-dataset-clean.csv"
df_cleaned = pd.read_csv(file_path)

In [86]:
df_cleaned.columns

Index(['contract-delivery', 'demand-forecast', 'temperature-normal',
       'temperature-forecast', 'solar-forecast', 'day-ahead-auction-price',
       'wind-forecast', 'day-ahead-auction-time', 'hour', 'day_of_week',
       'month', 'is_weekend', 'hour_sin', 'hour_cos', 'day_of_week_sin',
       'day_of_week_cos', 'month_sin', 'month_cos', 'lag_1', 'lag_7',
       'rolling_mean_7', 'rolling_std_7', 'wind_volatility',
       'solar_volatility', 'wind_solar_interaction',
       'demand_to_renewable_ratio', 'price_volatility'],
      dtype='object')

# Historical Average and Linear regression

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Define features and target variable
features = ["demand-forecast", 'temperature-normal', "solar-forecast", "wind-forecast", "temperature-forecast", "hour", "day_of_week", "month"]
target = "day-ahead-auction-price"

# Split data into train and test sets (80% train, 20% test)
train, test = train_test_split(df_cleaned, test_size=0.2, random_state=42, shuffle=False)

# Train a Linear Regression model
X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
test["lr_pred"] = lr_model.predict(X_test)


# Historical average baseline: Predict using the average price per hour from the training set
hourly_avg_price = train.groupby("hour")[target].mean()
test["historical_avg_pred"] = test["hour"].map(hourly_avg_price)

# Evaluation metrics
baseline_mae = mean_absolute_error(y_test, test["historical_avg_pred"])
baseline_rmse = np.sqrt(mean_squared_error(y_test, test["historical_avg_pred"]))

lr_mae = mean_absolute_error(y_test, test["lr_pred"])
lr_rmse = np.sqrt(mean_squared_error(y_test, test["lr_pred"]))

print(f"Historical Average Baseline - MAE: {baseline_mae:.2f}, RMSE: {baseline_rmse:.2f}")
print(f"Linear Regression Model - MAE: {lr_mae:.2f}, RMSE: {lr_rmse:.2f}")


Historical Average Baseline - MAE: 30.44, RMSE: 40.95
Linear Regression Model - MAE: 17.92, RMSE: 24.02


Linear Regression model is a big improvement over the historical average baseline! 🚀
Analysis of Results

    Baseline (Historical Average)
        MAE: 30.37 → On average, predictions are €30.37 off from actual prices.
        RMSE: 40.85 → Higher penalty for large errors, meaning occasional big misses.

    Linear Regression Model
        MAE: 17.70 → On average, predictions are now only €17.70 off (almost 50% improvement).
        RMSE: 23.81 → Reduced significantly, meaning fewer large errors.

How Good is This?

    Big improvement over baseline, meaning features are relevant.
    Still room for improvement: €17.70 is still a relatively high error for a forecasting model.

In [46]:
import numpy as np

# Ensure data is sorted correctly
df_cleaned = df_cleaned.sort_values("contract-delivery")

# Lag Feature: Previous day's price for the same hour
df_cleaned["lag_1"] = df_cleaned.groupby("hour")["day-ahead-auction-price"].shift(24)

# Rolling Statistics: 7-day moving average and standard deviation
df_cleaned["rolling_mean_7"] = df_cleaned["day-ahead-auction-price"].rolling(window=7*24, min_periods=1).mean()
df_cleaned["rolling_std_7"] = df_cleaned["day-ahead-auction-price"].rolling(window=7*24, min_periods=1).std()

# Renewable Ratio: (Solar + Wind) / Demand
df_cleaned["renewable_ratio"] = (df_cleaned["solar-forecast"] + df_cleaned["wind-forecast"]) / df_cleaned["demand-forecast"]

# Time-Based Features: Sinusoidal encoding for hour
df_cleaned["hour_sin"] = np.sin(2 * np.pi * df_cleaned["hour"] / 24)
df_cleaned["hour_cos"] = np.cos(2 * np.pi * df_cleaned["hour"] / 24)

# Drop rows with NaNs introduced by lag features
df_cleaned = df_cleaned.dropna()

# Display sample of new features
df_cleaned[["day-ahead-auction-price", "lag_1", "rolling_mean_7", "rolling_std_7", "renewable_ratio", "hour_sin", "hour_cos"]].head()


,day-ahead-auction-price,lag_1,rolling_mean_7,rolling_std_7,renewable_ratio,hour_sin,hour_cos
576,35.09,38.60,42.938095,10.850749,0.184487,0.000000,1.000000
577,37.01,36.55,43.026012,10.741326,0.180647,0.258819,0.965926
578,35.14,32.32,43.084226,10.671283,0.175985,0.500000,0.866025
579,33.11,30.85,43.130179,10.611084,0.166013,0.707107,0.707107
580,33.01,30.14,43.172143,10.556574,0.154348,0.866025,0.500000


## Re-run baseline model

In [47]:
# Updated feature set with new engineered features
features = [
    "demand-forecast", "solar-forecast", "wind-forecast", "temperature-forecast", 
    "hour", "day_of_week", "month", "lag_1", "rolling_mean_7", "rolling_std_7", 
    "renewable_ratio", "hour_sin", "hour_cos"
]

target = "day-ahead-auction-price"


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Split data into train and test sets (80% train, 20% test)
train, test = train_test_split(df_cleaned, test_size=0.2, random_state=42, shuffle=False)

# Train a Linear Regression model with new features
X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
test["lr_pred"] = lr_model.predict(X_test)

# Evaluate performance
lr_mae = mean_absolute_error(y_test, test["lr_pred"])
lr_rmse = np.sqrt(mean_squared_error(y_test, test["lr_pred"]))

print(f"Updated Linear Regression - MAE: {lr_mae:.2f}, RMSE: {lr_rmse:.2f}")


Updated Linear Regression - MAE: 16.24, RMSE: 21.42


<strong data-start="42" data-end="71">Comparison of Performance</strong>
<table data-start="74" data-end="334" node="[object Object]"><thead data-start="74" data-end="132"><tr data-start="74" data-end="132"><th data-start="74" data-end="82">Model</th><th data-start="82" data-end="106">MAE (Lower is Better)</th><th data-start="106" data-end="132">RMSE (Lower is Better)</th></tr></thead><tbody data-start="158" data-end="334"><tr data-start="158" data-end="200"><td><strong data-start="160" data-end="182">Historical Average</strong></td><td>30.37</td><td>40.85</td></tr><tr data-start="201" data-end="250"><td><strong data-start="203" data-end="232">Initial Linear Regression</strong></td><td>17.70</td><td>23.81</td></tr><tr data-start="251" data-end="334"><td><strong data-start="253" data-end="302">Updated Linear Regression (with new features)</strong></td><td><strong data-start="305" data-end="314">16.24</strong> 🔽</td><td><strong data-start="320" data-end="329">21.42</strong> 🔽</td></tr><table>

<strong data-start="340" data-end="357">Key Takeaways</strong>
<p data-start="358" data-end="547"><strong data-start="360" data-end="413">MAE improved from 17.70 → 16.24 (~8% improvement)</strong><br data-start="413" data-end="416">
<strong data-start="418" data-end="473">RMSE improved from 23.81 → 21.42 (~10% improvement)</strong><br data-start="473" data-end="476">
Feature engineering helped capture additional patterns in the data</p>

In [8]:
df_cleaned.to_csv('../data/clean_FeatEng.csv')

In [43]:
df_cleaned.columns

Index(['contract-delivery', 'demand-forecast', 'temperature-normal',
       'temperature-forecast', 'solar-forecast', 'day-ahead-auction-price',
       'wind-forecast', 'day-ahead-auction-time', 'wind_volatility',
       'solar_volatility', 'price_volatility', 'temperature_deviation'],
      dtype='object')

In [49]:
# Sort data by delivery time
df_cleaned = df_cleaned.sort_values(by="contract-delivery")

# Compute Rolling Volatility (std deviation over past 24 hours)
df_cleaned["wind_volatility"] = df_cleaned["wind-forecast"].rolling(window=24, min_periods=1).std()
df_cleaned["solar_volatility"] = df_cleaned["solar-forecast"].rolling(window=24, min_periods=1).std()
df_cleaned["price_volatility"] = df_cleaned["day-ahead-auction-price"].rolling(window=24, min_periods=1).std()

# Compute Weather Uncertainty (difference between forecast & normal temperature)
df_cleaned["temperature_deviation"] = np.abs(df_cleaned["temperature-forecast"] - df["temperature-normal"])

# Drop initial rows with NaNs (from rolling calculations)
df_cleaned = df_cleaned.dropna()

# Re-Train Linear Regression with Engineered Features

In [88]:
# Updated feature set with new engineered features
features = ['demand-forecast', 'temperature-normal',
       'temperature-forecast', 'solar-forecast',
       'wind-forecast', 'hour', 'day_of_week',
       'month', 'is_weekend', 'hour_sin', 'hour_cos', 'day_of_week_sin',
       'day_of_week_cos', 'month_sin', 'month_cos', 'lag_1', 'lag_7',
       'rolling_mean_7', 'rolling_std_7', 'wind_volatility',
       'solar_volatility', 'wind_solar_interaction',
       'demand_to_renewable_ratio']
target = "day-ahead-auction-price"


In [89]:
# Split data into train and test sets (80% train, 20% test)
train, test = train_test_split(df_cleaned, test_size=0.2, random_state=42, shuffle=False)

# Train a Linear Regression model with new features
X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]

lr_model = LinearRegression()
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
test["lr_pred"] = lr_model.predict(X_test)

# Evaluate performance
lr_mae = mean_absolute_error(y_test, test["lr_pred"])
lr_rmse = np.sqrt(mean_squared_error(y_test, test["lr_pred"]))

print(f"Updated Linear Regression - MAE: {lr_mae:.2f}, RMSE: {lr_rmse:.2f}")

Updated Linear Regression - MAE: 16.45, RMSE: 21.86


| Model                                            | Mae   | Rmse  |
| :----------------------------------------------- | :---- | :---- |
| Historical Average                               | 30.37 | 40.85 |
| Initial Linear Regression                        | 17.7  | 23.81 |
| **Improved Linear Regression (Feature Engineering)** | **16.24** | **21.42** |
